In [59]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [61]:
print('The train image dataset has shape:', train_images.shape)
print('The test image dataset has shape:',test_images.shape)
print('The train label dataset has shape:', len(train_labels))
print('The test label dataset has shape:',test_labels.shape)

The train image dataset has shape: (60000, 28, 28)
The test image dataset has shape: (10000, 28, 28)
The train label dataset has shape: 60000
The test label dataset has shape: (10000,)


normalize data

In [62]:
train_images = train_images / 255.0

change the datashet to 28x28=784 vector

In [63]:
size = 20000 # size of sample from the training data
nsamples, nx, ny = train_images.shape
train_images_vector = train_images.reshape((nsamples,nx*ny))
train_images_vector.shape
train_images_vector = train_images_vector[0:size]
train_images_vector.shape

(20000, 784)

initialize clusters vector to 0 and centroids vector to 0

In [64]:
X = train_images_vector
K = 10
clusters = np.full(size, 0)   
centroids = np.zeros((10, 784))

put initial values in centroids and clusters

In [65]:
indexes = np.random.choice(size, K, replace=False)
print(indexes)
for i in range(0, len(indexes)):
    centroids[i] = X[indexes[i]]
    clusters[indexes[i]] = i

[ 6314 16903 16632 10594 15650 15313 19703 13612 10565 16142]


# Kmeans with Euclidean Distance

In [66]:
temp_cluster = 0
for k in range(0, 10):
    for i in range(X.shape[0]):
        min_distance = np.sum(np.square(X[0] - centroids[0]))
        temp_cluster = 0                    
        for j in range(K):    
            if min_distance > np.sum(np.square(X[i] - centroids[j])):
                min_distance = np.sum(np.square(X[i] - centroids[j]))
                temp_cluster = j
                              
        clusters[i] = temp_cluster

    # find the middle of the cluster
    for i in range(K):
        data_index = np.where(clusters == i)
        if(len(data_index[0]) != 0):
#             print("class: ",  i)
#             print("indexes: ", data_index[0])
            sum_data = 0
            for pos in data_index[0]:
                sum_data = sum_data + X[pos]        
            centroids[i] = sum_data / len(data_index[0])
    
    

# Calculate Purity

In [67]:
train_labels = train_labels[0:size]
correct_predictions = np.where(clusters == train_labels)
purity = len(correct_predictions[0]) / size
purity


0.1335

# Calculate F- measure

In [69]:
all_f1_scores = []
for cluster_category in range(0, 10):
    current_cluster_index = np.where(clusters == cluster_category)

    true_current_labels = []
    for x in current_cluster_index[0]:
        true_current_labels.append(train_labels[x])

    # find pleiopsifiotita pou einai to positive
    freq = {}    
    for item in true_current_labels:  
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1

    positive = max(freq, key=freq.get)

    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in true_current_labels:
        if(i == cluster_category):
            if(i == positive):
                tp =tp + 1
            else:
                tn = tn + 1
        else:
            if(i == positive):
                fp =fp + 1
            else:
                fn = fn + 1
    
    if (tp + fp) != 0:
        precision = tp / (tp + fp)
    if (tp + fn) != 0:    
        recall = tp / (tp + fn)
    
    if(precision == recall == 0):
        f1 = 0
    else:   
        f1 = 2* (precision*recall) / (precision+recall)

    all_f1_scores.append(f1)
total_f1 = sum(all_f1_scores)
total_f1

0.7800361336946703